In [3]:
import numpy as np
import gym
import operator

In [24]:
class Store(gym.Env):
    def __init__(self):
        self.v_demand = [100, 200, 300, 400]
        self.p_demand = [0.3, 0.4, 0.2, 0.1]
        self.capacity = self.v_demand[-1]
        self.days = ['Mon', 'Tue', 'Wed',
                     'Thu', 'Fri', 'Sat', 'Sun']
        self.unit_cost = 4
        self.net_revenue = 7
        self.action_space = [0, 100, 200, 300, 400]
        self.state_space = [(d, i) for d in self.days for i in [0, 100, 200, 300, 400]]

    def get_next_state_reward(self, state, action, demand):
        day, inventory = state
        result = {}
        result['next_day'] = self.days[self.days.index(day) \
                                       + 1]
        result['starting_inventory'] = min(self.capacity,
                                           inventory
                                           + action)
        result['cost'] = self.unit_cost * action
        result['sales'] = min(result['starting_inventory'],
                              demand)
        result['revenue'] = self.net_revenue * result['sales']
        result['next_inventory'] \
            = result['starting_inventory'] - result['sales']
        result['reward'] = result['revenue'] - result['cost']
        return result

    def get_transition_prob(self, state, action):
        next_s_r_prob = {}
        for ix, demand in enumerate(self.v_demand):
            result = self.get_next_state_reward(state,
                                                action,
                                                demand)
            next_s = (result['next_day'],
                      result['next_inventory'])
            reward = result['reward']
            prob = self.p_demand[ix]
            if (next_s, reward) not in next_s_r_prob:
                next_s_r_prob[next_s, reward] = prob
            else:
                next_s_r_prob[next_s, reward] += prob
        return next_s_r_prob

    def reset(self):
        self.day = "Mon"
        self.inventory = 0
        state = (self.day, self.inventory)
        return state

    def is_terminal(self, state):
        day, inventory = state
        if day == "Sun":
            return True
        else:
            return False

    def step(self, action):
        demand = np.random.choice(self.v_demand,
                                  p=self.p_demand)
        result = self.get_next_state_reward((self.day,
                                             self.inventory),
                                       action,
                                       demand)
        self.day = result['next_day']
        self.inventory = result['next_inventory']
        state = (self.day, self.inventory)
        reward = result['reward']
        done = self.is_terminal(state)
        info = {'demand': demand, 'sales': result['sales']}
        return state, reward, done, info

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [25]:
store = Store()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [9]:
def choose_action(state, policy):
    prob_a = policy[state]
    action = np.random.choice(a=list(prob_a.keys()),
                              p=list(prob_a.values()))
    return action

def simulate_policy(policy, n_episodes):
    np.random.seed(0)
    store = Store()
    rewards = []
    for i_episode in range(n_episodes):
        state = store.reset()
        done = False
        ep_reward = 0
        while not done:
            action = choose_action(state, policy)
            state, reward, done, info = store.step(action)
            ep_reward += reward
        rewards.append(ep_reward)
    print("Expected weekly profit:", np.mean(rewards))

In [10]:
def get_trajectory(env, policy):
    trajectory = []
    state = env.reset()
    done = False
    sar = [state]
    while not done:
        action = choose_action(state, policy)
        state, reward, done, info = env.step(action)
        sar.append(action)
        sar.append(reward)
        trajectory.append(sar)
        sar = [state]
    return trajectory

In [11]:
def get_eps_greedy(actions, eps, a_best):
    prob_a = {}
    n_a = len(actions)
    for a in actions:
        if a == a_best:
            prob_a[a] = 1 - eps + eps/n_a
        else:
            prob_a[a] = eps/n_a
    return prob_a

In [5]:
def get_random_policy(states, actions):
    policy = {}
    n_a = len(actions)
    for s in states:
        policy[s] = {a: 1/n_a for a in actions}
    return policy

In [12]:
def on_policy_first_visit_mc(env, n_iter, eps, gamma):
    np.random.seed(0)
    states =  env.state_space
    actions = env.action_space
    policy =  get_random_policy(states, actions)
    Q = {s: {a: 0 for a in actions} for s in states}
    Q_n = {s: {a: 0 for a in actions} for s in states}
    for i in range(n_iter):
        if i % 10000 == 0:
            print("Iteration:", i)
        trajectory = get_trajectory(env, policy)
        G = 0
        T = len(trajectory) - 1
        for t, sar in enumerate(reversed(trajectory)):
            s, a, r = sar
            G = r + gamma * G
            first_visit = True
            for j in range(T - t):
                s_j = trajectory[j][0]
                a_j = trajectory[j][1]
                if (s, a) == (s_j, a_j):
                    first_visit = False
            if first_visit:
                Q[s][a] = Q_n[s][a] * Q[s][a] + G
                Q_n[s][a] += 1
                Q[s][a] /= Q_n[s][a]
                a_best = max(Q[s].items(),
                             key=operator.itemgetter(1))[0]
                policy[s] = get_eps_greedy(actions,
                                           eps,
                                           a_best)
    return policy, Q, Q_n

In [27]:
policy, Q, Q_n = on_policy_first_visit_mc(store,
                                          300000,
                                          0.05,
                                          1)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Iteration: 0
Iteration: 10000
Iteration: 20000
Iteration: 30000
Iteration: 40000
Iteration: 50000
Iteration: 60000
Iteration: 70000
Iteration: 80000
Iteration: 90000
Iteration: 100000
Iteration: 110000
Iteration: 120000
Iteration: 130000
Iteration: 140000
Iteration: 150000
Iteration: 160000
Iteration: 170000
Iteration: 180000
Iteration: 190000
Iteration: 200000
Iteration: 210000
Iteration: 220000
Iteration: 230000
Iteration: 240000
Iteration: 250000
Iteration: 260000
Iteration: 270000
Iteration: 280000
Iteration: 290000


In [28]:
policy

{('Mon', 0): {0: 0.01, 100: 0.01, 200: 0.01, 300: 0.01, 400: 0.96},
 ('Mon', 100): {0: 0.2, 100: 0.2, 200: 0.2, 300: 0.2, 400: 0.2},
 ('Mon', 200): {0: 0.2, 100: 0.2, 200: 0.2, 300: 0.2, 400: 0.2},
 ('Mon', 300): {0: 0.2, 100: 0.2, 200: 0.2, 300: 0.2, 400: 0.2},
 ('Mon', 400): {0: 0.2, 100: 0.2, 200: 0.2, 300: 0.2, 400: 0.2},
 ('Tue', 0): {0: 0.01, 100: 0.01, 200: 0.01, 300: 0.01, 400: 0.96},
 ('Tue', 100): {0: 0.01, 100: 0.01, 200: 0.01, 300: 0.96, 400: 0.01},
 ('Tue', 200): {0: 0.01, 100: 0.01, 200: 0.96, 300: 0.01, 400: 0.01},
 ('Tue', 300): {0: 0.96, 100: 0.01, 200: 0.01, 300: 0.01, 400: 0.01},
 ('Tue', 400): {0: 0.2, 100: 0.2, 200: 0.2, 300: 0.2, 400: 0.2},
 ('Wed', 0): {0: 0.01, 100: 0.01, 200: 0.01, 300: 0.01, 400: 0.96},
 ('Wed', 100): {0: 0.01, 100: 0.01, 200: 0.01, 300: 0.96, 400: 0.01},
 ('Wed', 200): {0: 0.01, 100: 0.01, 200: 0.96, 300: 0.01, 400: 0.01},
 ('Wed', 300): {0: 0.96, 100: 0.01, 200: 0.01, 300: 0.01, 400: 0.01},
 ('Wed', 400): {0: 0.2, 100: 0.2, 200: 0.2, 300: 0.

In [29]:
def off_policy_mc(env, n_iter, eps, gamma):
    np.random.seed(0)
    states =  env.state_space
    actions = env.action_space
    Q = {s: {a: 0 for a in actions} for s in states}
    C = {s: {a: 0 for a in actions} for s in states}
    target_policy = {}
    behavior_policy = get_random_policy(states,
                                        actions)
    for i in range(n_iter):
        if i % 10000 == 0:
            print("Iteration:", i)
        trajectory = get_trajectory(env,
                                    behavior_policy)
        G = 0
        W = 1
        T = len(trajectory) - 1
        for t, sar in enumerate(reversed(trajectory)):
            s, a, r = sar
            G = r + gamma * G
            C[s][a] += W
            Q[s][a] += (W/C[s][a]) * (G - Q[s][a])
            a_best = max(Q[s].items(),
                         key=operator.itemgetter(1))[0]
            target_policy[s] = a_best
            behavior_policy[s] = get_eps_greedy(actions,
                                                eps,
                                                a_best)
            if a != target_policy[s]:
                break
            W = W / behavior_policy[s][a]
    target_policy = {s: target_policy[s] for s in states
                                   if s in target_policy}
    return target_policy, Q

In [30]:
policy, Q = off_policy_mc(store, 300000, 0.05, 1)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Iteration: 0
Iteration: 10000
Iteration: 20000
Iteration: 30000
Iteration: 40000
Iteration: 50000
Iteration: 60000
Iteration: 70000
Iteration: 80000
Iteration: 90000
Iteration: 100000
Iteration: 110000
Iteration: 120000
Iteration: 130000
Iteration: 140000
Iteration: 150000
Iteration: 160000
Iteration: 170000
Iteration: 180000
Iteration: 190000
Iteration: 200000
Iteration: 210000
Iteration: 220000
Iteration: 230000
Iteration: 240000
Iteration: 250000
Iteration: 260000
Iteration: 270000
Iteration: 280000
Iteration: 290000


In [31]:
policy

{('Mon', 0): 400,
 ('Tue', 0): 400,
 ('Tue', 100): 300,
 ('Tue', 200): 200,
 ('Tue', 300): 100,
 ('Wed', 0): 400,
 ('Wed', 100): 300,
 ('Wed', 200): 200,
 ('Wed', 300): 100,
 ('Thu', 0): 400,
 ('Thu', 100): 300,
 ('Thu', 200): 200,
 ('Thu', 300): 100,
 ('Fri', 0): 300,
 ('Fri', 100): 200,
 ('Fri', 200): 100,
 ('Fri', 300): 0,
 ('Sat', 0): 200,
 ('Sat', 100): 100,
 ('Sat', 200): 0,
 ('Sat', 300): 0}